In [ ]:
import a1chemy.data_source as data_source
import a1chemy.plot.plotly_tools as plotly_tools
import pymongo
import pandas as pd

mongo_client = pymongo.MongoClient("mongodb://localhost:27017/", username='a1chemy', password='1B2C9046-E3CC-447F-9961-E125759BA44F')
mongo_ticks_client = data_source.MongoTicks(mongo_client)
mongo_fund = data_source.MongoFund(mongo_client=mongo_client)


In [ ]:
ticks = mongo_ticks_client.find_one(exchange='SH', symbol='SH688036')
ticks.raw_data['close'].max()
plotly_tools.ticks_thumbnail(ticks, start=-20)

In [ ]:
from a1chemy.util import *
import plotly.graph_objects as go
from IPython.display import Image
import plotly.io as pio
import base64

def thumbnail(ticks=None, start=None, end=None, width=120, height=30):
    df = ticks.raw_data.iloc[start:end]
    min = df['close'].min()
    max = df['close'].max()
    def _color_getter(x):
        if x == min:
            print("min")
            return "red"
        elif x == max:
            return "green"
        else:
            return "yellow"
    
    def _size_getter(x):
        if x == min or x == max:
            return 12
        else:
            return 12
    
    layout = go.Layout(autosize=True, margin={'l': 0, 'r': 0, 't': 0, 'b': 0}, xaxis = dict(type="category"))
    fig = go.Figure(layout=layout, data=[
                    go.Scatter(x=df['time'],
                               y=df['close'],
                               mode='markers+lines',
                               marker = dict(size=list(map(_size_getter, df['close'])), color=list(map(_color_getter, df['close']))),
                               )
                    ]
                    )
    fig.update_layout(xaxis_visible=False, yaxis_visible=False)
    return Image(pio.to_image(fig, format='png', engine="kaleido", width=width, height=height))

print(ticks.raw_data['time'].min())
print(ticks.raw_data['time'].min().tz)
print(type(ticks.raw_data['time'].min()))
thumbnail(ticks, start=-20, width=480, height=120)

In [ ]:
def fund_amount_thumbnail(fund=None, start=None, end=None, width=120, height=30):
    history = [d.to_dict() for d in fund.history]
    df = pd.DataFrame(history)
    df = df.iloc[start:end]
    df['amount_trend'] = df['amount'] - df['amount'].min()
    
    min = df['amount_trend'].min()
    max = df['amount_trend'].max()
    def _color_getter(x):
        if x == min:
            return "red"
        elif x == max:
            return "black"
        else:
            return "grey"
    layout = go.Layout(autosize=True, margin={'l': 0, 'r': 0, 't': 0, 'b': 0}, xaxis = dict(type="category"))
    fig = go.Figure(layout=layout, data=[
                    go.Bar(y=df['amount_trend'],
                          marker = dict(color=list(map(_color_getter, df['amount_trend']))))
                    ]
                    )
    fig.update_layout(xaxis_visible=False, yaxis_visible=False)
    return Image(pio.to_image(fig, format='png', engine="kaleido", width=width, height=height))

In [ ]:
fund = mongo_fund.find_one(exchange=None, symbol='SH512170')
fund_amount_thumbnail(fund=fund, start=-20, end=None, width=120, height=30)